In [ ]:
import numpy as np
import tensorflow
np.random.seed(0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
np.random.seed(1)

2024-10-02 19:29:24.440851: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 19:29:24.440910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 19:29:24.441944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 19:29:24.448447: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 19:29:25.160715: W tensorflow/compiler/tf2

In [ ]:
import csv
def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

In [ ]:
X_train, Y_train = read_csv('train_emoji.csv')
X_test, Y_test = read_csv('tesss.csv')

In [ ]:
X_train

array(['never talk to me again', 'I am proud of your achievements',
       'It is the worst day in my life', 'Miss you so much',
       'food is life', 'I love you mum', 'Stop saying bullshit',
       'congratulations on your acceptance',
       'The assignment is too long ', 'I want to go play',
       'she did not answer my text ', 'Your stupidity has no limit',
       'how many points did he score', 'my algorithm performs poorly',
       'I got approved', 'Stop shouting at me',
       'Sounds like a fun plan ha ha', 'no one likes him',
       'the game just finished', 'I will celebrate soon',
       'So sad you are not coming', 'She is my dearest love', 'Good job',
       'It was funny lol', 'candy is life ', 'The chicago cubs won again',
       'I am hungry', 'I am so excited to see you after so long',
       'you did well on you exam', 'lets brunch some day',
       'he is so cute', 'How dare you ask that',
       'do you want to join me for dinner ', 'I said yes',
       'she is 

In [ ]:
maxLen = len(max(X_train, key=lambda x: len(x.split())).split())

In [ ]:
label_to_emoji = {0: "❤️", 1: "⚾:", 2: "😊", 3: "😞", 4: "🍴"}

In [ ]:
for idx in range(10):
    print(X_train[idx], label_to_emoji[Y_train[idx]])

never talk to me again 😞
I am proud of your achievements 😊
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😊
The assignment is too long  😞
I want to go play ⚾:


In [ ]:

def sentences_to_indices(X, word_to_index, max_len):

    m = X.shape[0]
    X_indices = np.zeros((m, max_len))

    for i in range(m):
        sentence_words = sentence_words = X[i].lower().split()
        j = 0
        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                j =  j + 1
    return X_indices

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):

    vocab_size = len(word_to_index) + 1
    any_word = next(iter(word_to_vec_map.keys()))
    emb_dim = word_to_vec_map[any_word].shape[0]
    emb_matrix = np.zeros((vocab_size, emb_dim))
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
    embedding_layer = Embedding(vocab_size, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])

    return embedding_layer

In [ ]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):

    sentence_indices = sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    model = Model(sentence_indices, X)
    return model

In [ ]:

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [ ]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
                line = line.strip().split()
                curr_word = line[0]
                words.add(curr_word)
                word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
                words_to_index[w] = i
                index_to_words[i] = w
                i = i + 1
        return words_to_index, index_to_words, word_to_vec_map


In [ ]:
!pip install gensim

In [ ]:
from gensim.downloader import load

# Carregar o modelo Word2Vec do Gensim
model = load('word2vec-google-news-300')

# Acessar o vetor de uma palavra
word_vector = model['word']  # Exemplo: obter o vetor para a palavra "word"
print("Vetor para 'word':", word_vector)

# Acessar a similaridade entre duas palavras
similarity = model.similarity('king', 'queen')
print("Similaridade entre 'king' e 'queen':", similarity)

# Encontrar palavras mais similares a uma palavra dada
similar_words = model.most_similar('king', topn=5)
print("Palavras mais similares a 'king':", similar_words)

# Verifique se a palavra está no vocabulário
if 'king' in model:
    print("'king' está no vocabulário.")
else:
    print("'king' não está no vocabulário.")


Vetor para 'word': [ 3.59375000e-01  4.15039062e-02  9.03320312e-02  5.46875000e-02
 -1.47460938e-01  4.76074219e-02 -8.49609375e-02 -2.04101562e-01
  3.10546875e-01 -1.05590820e-02 -6.15234375e-02 -1.55273438e-01
 -1.52343750e-01  8.54492188e-02 -2.70996094e-02  3.84765625e-01
  4.78515625e-02  2.58789062e-02  4.49218750e-02 -2.79296875e-01
  9.09423828e-03  4.08203125e-01  2.40234375e-01 -3.06640625e-01
 -1.80664062e-01  4.73632812e-02 -2.63671875e-01  9.08203125e-02
  1.37695312e-01 -7.20977783e-04  2.67333984e-02  1.92382812e-01
 -2.29492188e-02  9.70458984e-03 -7.37304688e-02  4.29687500e-01
 -7.93457031e-03  1.06445312e-01  2.80761719e-02 -2.29492188e-01
 -1.91650391e-02 -2.36816406e-02  3.51562500e-02  1.71875000e-01
 -1.12304688e-01  6.25000000e-02 -1.69921875e-01  1.29882812e-01
 -1.54296875e-01  1.58203125e-01 -7.76367188e-02  1.78710938e-01
 -1.72851562e-01  9.96093750e-02  3.94531250e-01  6.44531250e-02
 -6.83593750e-02 -3.18359375e-01  5.95703125e-02 -1.02539062e-02
  9.37

In [ ]:
word_to_index = {word: index for index, word in enumerate(model.index_to_key)}
index_to_word = {index: word for index, word in enumerate(model.index_to_key)}
word_to_vec_map = {word: model[word] for word in model.index_to_key}

In [ ]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

2024-10-02 19:31:05.492857: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-02 19:31:05.539143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-02 19:31:05.539351: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 300)           900000300 
                                                                 
 lstm (LSTM)                 (None, 10, 128)           219648    
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 5)                 645   

In [ ]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, 5)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50


2024-10-02 19:31:29.527631: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-10-02 19:31:30.494759: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f08e43364b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-02 19:31:30.494801: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2024-10-02 19:31:30.523476: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1727897490.681411     223 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 6s 6ms/step - loss: 1.5854 - accuracy: 0.3030
Epoch 2/50
5/5 [==============================] - 0s 5ms/step - loss: 1.5002 - accuracy: 0.3485
Epoch 3/50
5/5 [==============================] - 0s 5ms/step - loss: 1.4457 - accuracy: 0.3182
Epoch 4/50
5/5 [==============================] - 0s 5ms/step - loss: 1.3281 - accuracy: 0.3788
Epoch 5/50
5/5 [==============================] - 0s 5ms/step - loss: 1.2212 - accuracy: 0.4773
Epoch 6/50
5/5 [==============================] - 0s 5ms/step - loss: 0.9552 - accuracy: 0.7121
Epoch 7/50
5/5 [==============================] - 0s 5ms/step - loss: 0.7349 - accuracy: 0.7652
Epoch 8/50
5/5 [==============================] - 0s 5ms/step - loss: 0.5246 - accuracy: 0.8636
Epoch 9/50
5/5 [==============================] - 0s 5ms/step - loss: 0.4420 - accuracy: 0.8561
Epoch 10/50
5/5 [==============================] - 0s 5ms/step - loss: 0.3615 - accuracy: 0.8712
Epoch 11/50
5/5 [==============================] -

In [ ]:
x_test = np.array(['lets play some ball'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(X_test_indices)
print(x_test[0] +' '+  label_to_emoji[np.argmax(model.predict(X_test_indices))])

[[5530.  172.   78.  589.    0.    0.    0.    0.    0.    0.]]
1/1 [==============================] - 0s 17ms/step
lets play some ball ⚾:
